# Terrier Learning to Rank Examples

This notebook demonstrates the use of Pyterrier in a learning-to-rank fashion.

## Preparation

Lets install pyterrier, as usual.

In [14]:
#!pip install python-terrier
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-e2c_k3ze/python-terrier
  Running command git clone -q https://github.com/terrier-org/pyterrier.git /tmp/pip-install-e2c_k3ze/python-terrier
  Created wheel for python-terrier: filename=python_terrier-0.3.0.dev0-cp36-none-any.whl size=37418 sha256=9fcedd75b4d85b9c026e34c93cd5c20167ed10636d545e3da9de41a545d003ba
  Stored in directory: /tmp/pip-ephem-wheel-cache-75s589ez/wheels/cc/bb/69/836d846a92c787b35ca6478119c0033762ab2b95d866eeb288
Successfully built python-terrier
  Found existing installation: python-terrier 0.3.0.dev0
    Uninstalling python-terrier-0.3.0.dev0:
      Successfully uninstalled python-terrier-0.3.0.dev0


## Init 

You must run pt.init() before other pyterrier functions and classes

Arguments:    
- `version` - terrier IR version e.g. "5.2"    
- `mem` - megabytes allocated to java e.g. 4096

In [15]:
import numpy as np
import pandas as pd
import pyterrier as pt
if not pt.started():
  pt.init()

## Load Files and Index

Again, lets focus on the small Vaswani test collection. Its easily accessible via the dataset API. 

In [16]:
dataset = pt.datasets.get_dataset("vaswani")

indexref = dataset.get_index()
topics = dataset.get_topics()
qrels = dataset.get_qrels()

## Multi-stage Retrieval

In this experiment, we will be re-ranking the results obtaind from a BM25 ranking, by adding more features. Will then pass these for re-ranking by a regression technique, such as Random Forests.

Conceptually, this pipeline has three stages:
1. PL2 ranking
2. Re-rank by each of the feaures ("TF_IDF" and "PL2")
3. Apply the RandomForests



In [17]:
#this ranker will make the candidate set of documents for each query
BM25 = pt.BatchRetrieve(indexref, controls = {"wmodel": "BM25"})

#these rankers we will use to re-rank the BM25 results
TF_IDF =  pt.BatchRetrieve(indexref, controls = {"wmodel": "TF_IDF"})
PL2 =  pt.BatchRetrieve(indexref, controls = {"wmodel": "PL2"})

OK, so how do we combine these?

In [18]:
pipe = BM25 >> (TF_IDF ** PL2)

Here, we are using two Pyterrer operators:
 - `>>` means "then", and takes the output documents of BM25 and puts them into the next stage. This means that TF_IDF and PL2 are ONLY applied on the documents that BM25 has identified.
 - `**` means feature-union - which makes each ranker into a feature in the `features` column of the results.

Lets give a look at the output to see what it gives:

In [19]:
pipe.transform("chemical end:2")

,qid,docid,docno,rank,score,query,docid_x,rank_x,query_x,docid_y,rank_y,query_y,features
0,1,10702,10703,0,13.472012,chemical end:2,10702,0,chemical end:2,10702,0,chemical end:2,"[7.38109017620895, 6.9992254918907575]"
1,1,1055,1056,1,12.517082,chemical end:2,1055,1,chemical end:2,1055,1,chemical end:2,"[6.857899681644975, 6.358419229871986]"
2,1,4885,4886,2,12.228161,chemical end:2,4885,2,chemical end:2,4885,2,chemical end:2,"[6.69960466053696, 6.181368165774688]"


See, we now have a "features" column with numbers representing the TF_IDF and PL2 feature scores.

*A note about efficiency*: doing retrieval, then re-ranking the documents again can be slow. For this reason, Terrier has a FeaturesBatchRetrieve. Lets try this:

In [20]:
fbr = pt.FeaturesBatchRetrieve(indexref, controls = {"wmodel": "BM25"}, features=["WMODEL:TF_IDF", "WMODEL:PL2"]) 
#lets look at the top 2 results
(fbr %2).transform("chemical")

,qid,docid,rank,docno,score,features
0,1,10702,0,10703,13.472012,"[7.38109017620895, 6.9992254918907575]"
1,1,1055,1,1056,12.517082,"[6.857899681644975, 6.358419229871986]"
2,1,4885,2,4886,12.228161,"[6.69960466053696, 6.181368165774688]"


However, this kind of optimisation is common in Pyterrier, so Pyterrier actually supports automatic pipeline optimisation, using the `.compile()` function.

In [21]:
pipe_fast = pipe.compile()
(pipe_fast %2).transform("chemical")

Applying 8 rules


,qid,docid,rank,docno,score,features
0,1,10702,0,10703,13.472012,"[7.38109017620895, 6.9992254918907575]"
1,1,1055,1,1056,12.517082,"[6.857899681644975, 6.358419229871986]"
2,1,4885,2,4886,12.228161,"[6.69960466053696, 6.181368165774688]"


Finally, often we want our initial retrieval score to be a feature also. We can do this in one of two ways:
 - by adding a `SAMPLE` feature to FeaturesBatchRetrieve
 - or in the original feature-union definition, including an IdentityTransformer 

In [22]:
fbr = pt.FeaturesBatchRetrieve(indexref, controls = {"wmodel": "BM25"}, features=["SAMPLE", "WMODEL:TF_IDF", "WMODEL:PL2"]) 
pipe = BM25 >> (pt.transformer.IdentityTransformer() ** TF_IDF ** PL2)

(pipe %2).transform("chemical")

,qid,docid,docno,rank,score_x,query,docid_x,rank_x,query_x,docid_y,rank_y,score_y,query_y,features
0,1,10702,10703,0,13.472012,chemical,10702,0,chemical,10702,0,13.472012,chemical,"[13.472012496423268, 7.38109017620895, 6.99922..."
1,1,1055,1056,1,12.517082,chemical,1055,1,chemical,1055,1,12.517082,chemical,"[12.517081895047532, 6.857899681644975, 6.3584..."
2,1,4885,4886,2,12.228161,chemical,4885,2,chemical,4885,2,12.228161,chemical,"[12.22816082084599, 6.69960466053696, 6.181368..."


# Learning models and re-ranking

Ok, lets get onto the actual machine learning. We can use standard Python ML techniques. We will demonstrate a few here, including from sci-kit learn and xgBoost.

In each case, the pattern is the same:
 - Create a transformer that does the re-ranking
 - Call the fit() method on the created object with the training topics (and validation topics as necessary)
 - Evaluate the results with the Experiment function by using the test topics

 Firstly, lets separate our topics into train/validation/test.

In [23]:
train_topics, valid_topics, test_topics = np.split(topics, [int(.6*len(topics)), int(.8*len(topics))])

## sci-kit learn RandomForestRegressor

Our first learning-to-rank will be done using sci-kit learn's [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html). 

We use `pt.piptlines.LTR_pipeline`, which is a pyterrier transformer that passes the document features as "X" features to RandomForest. To learn the model (called fitting) the RandomForest, we invoke the `fit()` method - on the entire pipeline, specifying the queries (topics) and relevance assessment (qrels). The latter for the "Y" labels for the RandomForest fitting.

NB: due to their bootstrap nature, Random Forests do not overfit, so we do not provide validation data to `fit()`.

On the other hand, we could use any regression learner from sklearn, and adjust its parameters ourselves.

Finally, we Experiment() on the test data to compare performances.

In [24]:
from sklearn.ensemble import RandomForestRegressor

BaselineLTR = fbr >> pt.pipelines.LTR_pipeline(RandomForestRegressor(n_estimators=400))
BaselineLTR.fit(train_topics, qrels)

results = pt.pipelines.Experiment([PL2, BaselineLTR], test_topics, qrels, ["map"], names=["PL2 Baseline", "LTR Baseline"])
results

,name,map
0,PL2 Baseline,0.206031
1,LTR Baseline,0.144980


## XgBoost Pipeline

We now demonstrate the use of a LambdaMART implementation from [xgBoost](https://xgboost.readthedocs.io/en/latest/). Again, pyTerrier provides a transformer object, namely `XGBoostLTR_pipeline`, which takes in the constrcutor the actual xgBoost model that you want to train. We took the xgBoost configuration from [their example code](https://github.com/dmlc/xgboost/blob/master/demo/rank/rank.py).

Call the `fit()` method on the full pipeline with the training and validation topics.

Evaluate the results with the Experiment function by using the test topics

In [25]:
import xgboost as xgb
params = {'objective': 'rank:ndcg', 
          'learning_rate': 0.1, 
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6,
          'verbose': 2,
          'random_state': 42 
         }

BaseLTR_LM = fbr >> pt.pipelines.XGBoostLTR_pipeline(xgb.sklearn.XGBRanker(**params))
BaseLTR_LM.fit(train_topics, qrels, valid_topics, qrels)

And evaluate the results.

In [26]:
allresultsLM = pt.pipelines.Experiment([PL2, BaseLTR_LM],
                                test_topics,                                  
                                qrels, ["map"], 
                                names=["PL2 Baseline", "LambdaMART"])
allresultsLM

,name,map
0,PL2 Baseline,0.206031
1,LambdaMART,0.204391
